# Uruchominie jointa w trybie torque

In [1]:
import can
import matplotlib as plt
import numpy as np
import os
import time
import math
from IPython.display import display, clear_output
from joint_can import *
%matplotlib inline


In [2]:
def initialize_joint(bus):
    send_msg(0x0A0, None, bus)
    time.sleep(1)
    
    send_msg(0x0b0, [0,0,0,0,0,0], bus)
    time.sleep(0.1)
    send_msg(0x040, None, bus)
    time.sleep(0.1)
    send_msg(0x0a0, None, bus)
    time.sleep(1)
    
    if (send_msg(0x0F0, [1, 2], bus).data[0] == 1):
        send_msg(0x010, [2], bus)
        time.sleep(1)
        print(send_msg(0x010, [3], bus).data[0])
        time.sleep(5)
        return True
    else:
        return False
     

In [3]:
if __name__ == "__main__":

    bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=1000000, data_bitrate=5000000, fd=False)
    initialize_joint(bus)
    
    time.sleep(5)

1


# Ustawienie torque'a

In [4]:
tq=16. #torque w Nm
duration = 300 #czas trwania w sekundach

# Blok główny programu

In [5]:
#blok pracuje w pętli, aby zatrzymać jointa trzeba zatrzymać ten blok i uruchomić kolejny

# fig = plt.pyplot.figure()
# ax = fig.add_subplot(1, 1, 1) 

spin=1
it=0
tq_data=[]

start=time.time()
elapsed=0
min_speed=2.
max_speed=0.
set_torque(tq,bus)
time.sleep(5)
#torque zwracany co 1s w Nm
while(elapsed<duration):
    elapsed = time.time()-start
    data=set_torque(tq,bus)
    tq_data.append(data.torque)
    min_speed=min(min_speed,data.speed)
    max_speed=max(max_speed,data.speed)
    print('\r speed: {0:.3} (min: {3:.3} | max: {4:.3})  | time: {1}s / {2}s      '.format(data.speed, int(elapsed),duration, min_speed, max_speed), end='')
    time.sleep(1)
    # ax.cla()
    # ax.plot(it, data.torque)
    # display(fig)
    # clear_output(wait = True)
    # it+=1
    # plt.pyplot.pause(5)
set_torque(0.,bus)

TypeError: set_torque() missing 1 required positional argument: 'id'

# Zatrzymanie jointa

In [ ]:
set_torque(0.,bus)    #zatrzymanie ctrl-enter